# Solutions - DO NOT POST
# Groupby and Arrest Data

In our merging exercises, we examined the relationship between county-level violent arrest totals and county-level drug arrest totals. In those exercises, you were given a dataset that provided you with county-level arrest totals. But that's not actually how the data are provided by the state of California. This week we will work with the *raw* California arrest data, which are not organized by county or even county-year. 

## Exercise 1: Learning the Group Structure of Your Data

**(1)** Load the data provided (Arrests.csv) which was downloaded from the California State Attorney General's office [here](https://openjustice.doj.ca.gov/data) (under "Arrests").

In [1]:
import pandas as pd
import numpy as np

pd.set_option("mode.copy_on_write", True)

file = "data/Arrests.csv"
df_arrests = pd.read_csv(file)
df_arrests

,YEAR,GENDER,RACE,AGE_GROUP,COUNTY,VIOLENT,PROPERTY,F_DRUGOFF,F_SEXOFF,F_ALLOTHER,F_TOTAL,M_TOTAL,S_TOTAL
0,1980,Male,Black,Under 18,Alameda County,505,1351,188,26,79,2149,2286,295
1,1980,Male,Black,18 to 19,Alameda County,205,465,183,8,48,909,1333,0
2,1980,Male,Black,20 to 29,Alameda County,949,1593,606,27,178,3353,7974,0
3,1980,Male,Black,30 to 39,Alameda County,450,755,241,18,110,1574,4876,0
4,1980,Male,Black,40 to 69,Alameda County,172,218,117,11,66,584,3836,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
104646,2022,Female,White,70 and over,Tulare County,1,0,0,0,0,1,6,0
104647,2022,Female,White,70 and over,Tuolumne County,1,0,0,0,0,1,5,0
104648,2022,Female,White,70 and over,Ventura County,1,1,0,0,0,2,46,0
104649,2022,Female,White,70 and over,Yolo County,1,0,0,0,0,1,4,0


In [2]:
df_arrests.groupby(by=["RACE", "COUNTY"], as_index=False)[["VIOLENT", "PROPERTY"]].sum()

,RACE,COUNTY,VIOLENT,PROPERTY
0,Black,Alameda County,101893,154701
1,Black,Alpine County,5,4
2,Black,Amador County,77,75
3,Black,Butte County,2068,1691
4,Black,Calaveras County,115,72
...,...,...,...,...
227,White,Tulare County,21557,29773
228,White,Tuolumne County,5023,7367
229,White,Ventura County,30149,44902
230,White,Yolo County,11624,18036


**(2)** What is the unit of observation for this dataset? In other words, when row zero says that there were 505 arrests for `VIOLENT` crimes, what exactly is that telling you -- 505 arrests in 1980? 505 arrests in Alameda County? Or is the data broken down even further?

**(3)** Using what we discussed in the reading on "Checking for Duplicates", use `duplicated` to test if the variables *you* think uniquely identify rows in your data really do uniquely identify rows. If you were wrong, update your understanding of the data!

In [3]:
df_arrests.duplicated(["COUNTY"]).any()

np.True_

**(4)** Once you have a handle on how the data are organized now, please **collapse the data** (using `groupby`) to be one observation per county-year-racial group. Make sure that you have the total arrest for violent crimes and the total arrests property crimes in your dataset.

**Note:** As mentioned in a previous reading, by default, `pandas` likes to make your grouping variables into a hierarchical index. Personally, I find hierarchical indices very weird and not worth dealing with. To avoid this, use the `as_index=False` option in `groupby`. I recommend this as a default option for working with `groupby`.

In [4]:
df_grouped = df_arrests.groupby(by=["COUNTY", "YEAR", "RACE"], as_index=False)[
    ["VIOLENT", "PROPERTY"]
].sum()
df_grouped

,COUNTY,YEAR,RACE,VIOLENT,PROPERTY
0,Alameda County,1980,Black,2594,5138
1,Alameda County,1980,Hispanic,592,903
2,Alameda County,1980,Other,139,233
3,Alameda County,1980,White,1179,3103
4,Alameda County,1981,Black,2753,5533
...,...,...,...,...,...
9928,Yuba County,2021,White,215,123
9929,Yuba County,2022,Black,61,7
9930,Yuba County,2022,Hispanic,58,32
9931,Yuba County,2022,Other,23,15


## Exercise 2: Investigating a question based on the data

In this section we'll work on the following question: does the racial composition of arrests in each county vary by arrest type? In other words, do Blacks make up a larger portion of the people arrested for drug offenses than violent offenses? To answer this question, you will need to compute the proportion of all arrests in a county-year that occur within each racial group. 

**(5)** We'll take this in steps. We want to get the fraction of violent crimes that correspond to each race and fo the same for property crimes AND we want to have these quantities for each county and each year. Before we can get fractions of a total, we need to calculate the total. Begin by creating a new column of data called "violent_total" that includes for each county and year sums violent crime arrests across all races. Hint: use `transform` to help you out.

You should be able to check your answer - 'violent_total' should be equal to the sum across all races for a given county and given year.

In [5]:
df_grouped["violent_total"] = df_grouped.groupby(["COUNTY", "YEAR"], as_index=False)[
    "VIOLENT"
].transform(lambda x: sum(x))
df_grouped

,COUNTY,YEAR,RACE,VIOLENT,PROPERTY,violent_total
0,Alameda County,1980,Black,2594,5138,4504
1,Alameda County,1980,Hispanic,592,903,4504
2,Alameda County,1980,Other,139,233,4504
3,Alameda County,1980,White,1179,3103,4504
4,Alameda County,1981,Black,2753,5533,4699
...,...,...,...,...,...,...
9928,Yuba County,2021,White,215,123,355
9929,Yuba County,2022,Black,61,7,327
9930,Yuba County,2022,Hispanic,58,32,327
9931,Yuba County,2022,Other,23,15,327


**(6)** Repeat this process to create a new column of data called "property_total" that includes for each county and year sums property crime arrests across all races. Hint: once again use `transform` to help you out and check that "property_total" equals the sum of property crime arrests across all races for a given county and given year.

In [6]:
df_grouped["property_total"] = df_grouped.groupby(["COUNTY", "YEAR"], as_index=False)[
    "PROPERTY"
].transform(lambda x: sum(x))
df_grouped

,COUNTY,YEAR,RACE,VIOLENT,PROPERTY,violent_total,property_total
0,Alameda County,1980,Black,2594,5138,4504,9377
1,Alameda County,1980,Hispanic,592,903,4504,9377
2,Alameda County,1980,Other,139,233,4504,9377
3,Alameda County,1980,White,1179,3103,4504,9377
4,Alameda County,1981,Black,2753,5533,4699,10014
...,...,...,...,...,...,...,...
9928,Yuba County,2021,White,215,123,355,181
9929,Yuba County,2022,Black,61,7,327,144
9930,Yuba County,2022,Hispanic,58,32,327,144
9931,Yuba County,2022,Other,23,15,327,144


## Exercise 3: Trying it on your own

**(7)** Now we have what we need to determine the fraction of violent and property crime arrests by race for each county and year. Add two new columns called "race_percent_violent" and "race_percent_property". To get the first, calculate each as the fraction of violent crime arrests by race divided by the total of violent crime arrests by race, multiplied by 100. Do a similar computation for "race_percent_property".

Always be sure to check your data to make sure the computation you were expecting worked - for example, percentages should add up to 100.

In [7]:
df_grouped["race_percent_violent"] = (
    df_grouped["VIOLENT"] / df_grouped["violent_total"] * 100
)
df_grouped["race_percent_property"] = (
    df_grouped["PROPERTY"] / df_grouped["property_total"] * 100
)
df_grouped

,COUNTY,YEAR,RACE,VIOLENT,PROPERTY,violent_total,property_total,race_percent_violent,race_percent_property
0,Alameda County,1980,Black,2594,5138,4504,9377,57.593250,54.793644
1,Alameda County,1980,Hispanic,592,903,4504,9377,13.143872,9.629946
2,Alameda County,1980,Other,139,233,4504,9377,3.086146,2.484803
3,Alameda County,1980,White,1179,3103,4504,9377,26.176732,33.091607
4,Alameda County,1981,Black,2753,5533,4699,10014,58.586933,55.252646
...,...,...,...,...,...,...,...,...,...
9928,Yuba County,2021,White,215,123,355,181,60.563380,67.955801
9929,Yuba County,2022,Black,61,7,327,144,18.654434,4.861111
9930,Yuba County,2022,Hispanic,58,32,327,144,17.737003,22.222222
9931,Yuba County,2022,Other,23,15,327,144,7.033639,10.416667


*A quick note of warning on interpretation: these results can tell you whether Black Californians make up a larger proportion of *arrests* for certain types of crimes, not whether they make up a larger proportion of people who *commit* a given type of crime! For example, there is extensive data that shows that Black and White Americans *use* drugs at the same rate, but Black Americans are arrested for drug use *much* more often. So be aware that arrests != crimes committed.*

**(8)** Let's say we wanted to identify the percentage of arrests by race for violent and property crime across ALL counties in California and compare that breakdown in 1980 to 2022. Repeat the analysis you completed above and modify it to gather this new set of information.

In [8]:
df = df_arrests.groupby(by=["YEAR", "RACE"], as_index=False)[
    ["VIOLENT", "PROPERTY"]
].sum()
df

,YEAR,RACE,VIOLENT,PROPERTY
0,1980,Black,29626,48563
1,1980,Hispanic,24605,42282
2,1980,Other,2327,3738
3,1980,White,29336,83829
4,1981,Black,30263,52220
...,...,...,...,...
167,2021,White,24343,17902
168,2022,Black,22309,11211
169,2022,Hispanic,43864,27975
170,2022,Other,6979,3737


In [9]:
df["violent_total"] = df.groupby(["YEAR"], as_index=False)["VIOLENT"].transform(
    lambda x: sum(x)
)
df["property_total"] = df.groupby(["YEAR"], as_index=False)["PROPERTY"].transform(
    lambda x: sum(x)
)
df

,YEAR,RACE,VIOLENT,PROPERTY,violent_total,property_total
0,1980,Black,29626,48563,85894,178412
1,1980,Hispanic,24605,42282,85894,178412
2,1980,Other,2327,3738,85894,178412
3,1980,White,29336,83829,85894,178412
4,1981,Black,30263,52220,86254,184507
...,...,...,...,...,...,...
167,2021,White,24343,17902,93728,58245
168,2022,Black,22309,11211,98115,62131
169,2022,Hispanic,43864,27975,98115,62131
170,2022,Other,6979,3737,98115,62131


In [10]:
df["race_percent_violent"] = df["VIOLENT"] / df["violent_total"] * 100
df["race_percent_property"] = df["PROPERTY"] / df["property_total"] * 100
df

,YEAR,RACE,VIOLENT,PROPERTY,violent_total,property_total,race_percent_violent,race_percent_property
0,1980,Black,29626,48563,85894,178412,34.491350,27.219582
1,1980,Hispanic,24605,42282,85894,178412,28.645773,23.699079
2,1980,Other,2327,3738,85894,178412,2.709153,2.095151
3,1980,White,29336,83829,85894,178412,34.153724,46.986189
4,1981,Black,30263,52220,86254,184507,35.085909,28.302449
...,...,...,...,...,...,...,...,...
167,2021,White,24343,17902,93728,58245,25.971961,30.735685
168,2022,Black,22309,11211,98115,62131,22.737604,18.044133
169,2022,Hispanic,43864,27975,98115,62131,44.706722,45.025833
170,2022,Other,6979,3737,98115,62131,7.113082,6.014711


In [11]:
df.loc[df["YEAR"] == 1980]

,YEAR,RACE,VIOLENT,PROPERTY,violent_total,property_total,race_percent_violent,race_percent_property
0,1980,Black,29626,48563,85894,178412,34.491350,27.219582
1,1980,Hispanic,24605,42282,85894,178412,28.645773,23.699079
2,1980,Other,2327,3738,85894,178412,2.709153,2.095151
3,1980,White,29336,83829,85894,178412,34.153724,46.986189


In [12]:
df.loc[df["YEAR"] == 2022]

,YEAR,RACE,VIOLENT,PROPERTY,violent_total,property_total,race_percent_violent,race_percent_property
168,2022,Black,22309,11211,98115,62131,22.737604,18.044133
169,2022,Hispanic,43864,27975,98115,62131,44.706722,45.025833
170,2022,Other,6979,3737,98115,62131,7.113082,6.014711
171,2022,White,24963,19208,98115,62131,25.442593,30.915324


**From your results, note the percent of violent crime associated with Black Californians in both 1980 and in 2022. Also note which race accounted for the highest percent of property crime in 1980 and in 2022. These values will be answers for your final quiz this week**

Note: echoing the caveat that these results can tell you whether any one race of Californians make up a larger proportion of *arrests* for certain types of crimes, not whether they make up a larger proportion of people who *commit* a given type of crime. Further analysis would be needed to explore those questions properly